In [ ]:
#概要
"""
住宅の価格を特徴量から予測する

評価方法 RMSE
"""

In [64]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")


In [65]:
train.head(5)
#test.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [66]:
train.describe()
#test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [67]:
print("data size \ntrain : {}\ntest : {}".format(len(train), len(test)))

data size 
train : 1460
test : 1459


In [68]:
#trainを学習用とValidation用に分ける 1000 : 460
train_size = 1000
train_data = train[:train_size]
val_data = train[train_size:]

In [69]:
#データの詳細を確認する

#データの種類の確認
#以下の3種類に分ける
#1. categorical data
#2. value data
#3. no relevance data


print("feature num", len(train.columns))
train.columns

feature num 81


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [70]:
#数値データと名前データに分ける
columns = train.columns
val_col = []
name_col = []

for c in columns:
    data = train[c]
    for d in data:
        #print(d, type(d))
        if type(d) == str:
            name_col.append(c)
            break
        if math.isnan(d):
            continue
        else:
            val_col.append(c)
            break
            

print(name_col, val_col)
print("val num :{} \nname num :{}".format(len(val_col), len(name_col)))
print("total num", len(val_col) + len(name_col))
print("base num", len(columns))



['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'] ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', '

In [112]:
#相関分析
#val data は相関を求める
#categorical data は 相関比を求める
#(cat * catは連関係数の算出 あるいは独立性検定?)
#2つの分布の比較（検定に）コルモゴロフーミルノフ検定というものがある

target = train["SalePrice"].copy()
val_corr = {}
cat_corr = {}

#val
#nanはmedian埋めをする
for vn in val_col:
    if vn == "SalePrice":
        continue
    data = train[vn].copy()
    median = data.median()
    data = data.fillna(median)
    corr = np.corrcoef(data, target)
    val_corr[vn] = corr[0][1]
    
print(val_corr)
#categorical
#相関比の計算を行う
#相関比 eta^2 = (カテゴリーごとに平均に変えた場合の全体の変動)/(全体の変動) \in [0, 1]
#相関比は１に近いほどカテゴリーごとの違いが大きくなる
#nanはNA categoryとする
for cn in name_col:
    data = train[cn].copy()
    data[data != data] = "NA"
    cat = data.unique()
    total_err = len(target) * np.var(target)
    def category_err(cname):
        ctarget = target[data==cname].copy()
        err = len(ctarget) * ((target.mean() - ctarget.mean())**2)
        return err
    cat_err = np.array([category_err(c) for c in cat])
    eta = np.sqrt(np.sum(cat_err) / total_err)
    cat_corr[cn] = eta
    
print(cat_corr)

{'Id': -0.021916719443431106, 'MSSubClass': -0.084284135126595189, 'LotFrontage': 0.33477085313976007, 'LotArea': 0.26384335387140562, 'OverallQual': 0.79098160058380496, 'OverallCond': -0.077855894048678015, 'YearBuilt': 0.52289733287949691, 'YearRemodAdd': 0.50710096711138619, 'MasVnrArea': 0.47261449900457386, 'BsmtFinSF1': 0.38641980624215339, 'BsmtFinSF2': -0.011378121450215144, 'BsmtUnfSF': 0.21447910554696892, 'TotalBsmtSF': 0.61358055155919544, '1stFlrSF': 0.6058521846919146, '2ndFlrSF': 0.31933380283206786, 'LowQualFinSF': -0.025606130000679544, 'GrLivArea': 0.70862447761265224, 'BsmtFullBath': 0.22712223313149427, 'BsmtHalfBath': -0.016844154297359009, 'FullBath': 0.56066376274844609, 'HalfBath': 0.28410767559478262, 'BedroomAbvGr': 0.16821315430073999, 'KitchenAbvGr': -0.13590737084214122, 'TotRmsAbvGrd': 0.53372315558202821, 'Fireplaces': 0.4669288367515278, 'GarageYrBlt': 0.46675365236333943, 'GarageCars': 0.64040919725835221, 'GarageArea': 0.62343143891836172, 'WoodDeckSF

feature num 81


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [107]:
#特に何も考えず線形回帰 or xgboost回帰により住宅価格の予測を行う

a = np.array([1,2,3,4,5])
np.var(a)*5 

10.0

In [108]:
np.sum((a - 3)**2)

10